In [2]:
from random import random, randint, choice
import numpy as np
from math import exp
from functools import partial
import multiprocess as mp

In [3]:
def run_iteration(R,alpha,beta,A,T,uni_mode,starting_Q,random_starting_Q,iter_num):
    import numpy as np
    from random import random, randint, choice
    print('a')
    if random_starting_Q:
        #Q = np.array([random()*(np.max(R)-np.min(R))+np.min(R),random()*(np.max(R)-np.min(R))+np.min(R)])
        Q = np.random.rand(2,2) * (np.max(R)-np.min(R))+np.min(R)
    else:
        Q = np.array([starting_Q,starting_Q])
    starting_Q = Q.copy()
#    prev_Q = Q.copy()
#    MaxQ = 0.0
    ArgmaxQ = [[0,0],[0,0]]
#    prev_argmaxQ = ArgmaxQ.copy()
    a=[0,0]
    t_last_change = 0
    exp_prob = 1
    for t in range(T):
        for p in range(2):

            # experiment
            #exp_prob = exp(-t*beta)
            if uni_mode:
                exp_prob = (0.015**(2/T))**t

                rdraw = random()
                if rdraw <= exp_prob:
                    a[p] = randint(0,A-1)
                else:
                    a[p] = choice(ArgmaxQ[p])
            else:
                beta = (0.015**(2/T))**t
                exps = np.exp(Q[p]/beta)
                prs = exps / np.sum(exps)

                rdraw = random()
                if rdraw <= prs[0]:
                    a[p]=0
                else:
                    a[p]=1

        for p in range(2):
            prev_Q = Q.copy()
            Q[p,a[p]] = alpha * Q[p,a[p]] + (1-alpha) * R[a[p],a[1-p]]
            MaxQ = np.max(Q,1)
            prev_argmaxQ = ArgmaxQ.copy()

            ArgmaxQ[p] = [x for x in range(A) if Q[p,x] == MaxQ[p]]

            if prev_argmaxQ[p]!= ArgmaxQ[p]:
            #if (prev_Q[p]!= Q[p]).any():
                t_last_change = t

        if t - t_last_change > 10000:
            print("converged")
            break

        #if t % (T/100) == 0:
        #    print(t/T*100,"% ",a,exp_prob,Q)
    if t >= T-1:
        print("did not converge")
    return [t,ArgmaxQ,starting_Q]

In [4]:
# T = 100000 # Maximum number of periods
# A = 2 # Number of actions
#
# alpha = 0.9
# beta = 0.9
#
# # Actions: 0 - Defect, 1 - Cooperate
# R = np.zeros([2,2])
# R[0,0] = 2 #DD
# R[1,0] = 1 #CD
# R[0,1] = 4 #DC
# R[1,1] = 3 #CC
#
#
# #print(Q)
# #print(t)
#
#
# alpha_list = [x / 10.0 for x in range(1, 9, 5)]
# r_list = [x / 10.0 for x in range(11, 29, 5)]
# #alpha_list = [0.9]
# #r_list = [1.2]
# num_iter = 5
# uni_mode = False
# converged_to = -np.ones([len(alpha_list),len(r_list),num_iter])
# conv_time = -np.ones([len(alpha_list),len(r_list),num_iter])
# for a_ind,alpha in enumerate(alpha_list):
#     for r_ind,r_dd in enumerate(r_list):
#         for iter in range(num_iter):
#             R[0,0]=r_dd
#             [t,conv_acts] = run_iteration(R,alpha,beta,A,T,uni_mode,starting_Q,random_starting_Q,iter)
#             if conv_acts == [[0],[0]]:
#                 converged_to[a_ind,r_ind,iter] = 0
#             elif conv_acts == [[1],[1]]:
#                 converged_to[a_ind,r_ind,iter] = 1
#             else:
#                 converged_to[a_ind,r_ind,iter] = -1
#             conv_time[a_ind,r_ind,iter] = t

In [ ]:
T = 100000 # Maximum number of periods
A = 2 # Number of actions

alpha = 0.9
beta = 0.9

# Actions: 0 - Defect, 1 - Cooperate
R = np.zeros([2,2])
R[0,0] = 2 #DD
R[1,0] = 1 #CD
R[0,1] = 4 #DC
R[1,1] = 3 #CC


#print(Q)
#print(t)


alpha_list = [x / 10.0 for x in range(1, 10, 1)]
r_list = [x / 10.0 for x in range(11, 30, 1)]
#alpha_list = [0.9]
#r_list = [1.2]
num_iter = 10
uni_mode = False
converged_to = -np.ones([len(alpha_list),len(r_list),num_iter])
conv_time = -np.ones([len(alpha_list),len(r_list),num_iter])
starting_Q = [float(R[0,0]),float(R[1,1])]
random_starting_Q = True
#starting_Q = [R[0,0],R[1,0]]

with mp.Pool() as pool: #defaults to number of available CPU's
#chunksize = 20 #this may take some guessing ... take a look at the docs to decide
    for a_ind,alpha in enumerate(alpha_list):
        for r_ind,r_dd in enumerate(r_list):
            R[0,0]=r_dd
            func = partial(run_iteration,R,alpha,beta,A,T,uni_mode,starting_Q,random_starting_Q)

            for iter, res in enumerate(pool.map(func, range(num_iter))):#, chunksize)):

                [t,conv_acts,first_Q] = res

                if conv_acts == [[0],[0]]:
                    converged_to[a_ind,r_ind,iter] = 0
                elif conv_acts == [[1],[1]]:
                    converged_to[a_ind,r_ind,iter] = 1
                else:
                    converged_to[a_ind,r_ind,iter] = -1
                conv_time[a_ind,r_ind,iter] = t

                print("done: ",alpha,r_dd,iter,"result: ",converged_to[a_ind,r_ind,iter],"time: ",t,"start Q: ",first_Q)


done:  0.1 1.1 0 result:  1.0 time:  27788 start Q:  [[2.37640924 3.16583378]
 [2.32833355 3.5700539 ]]
done:  0.1 1.1 1 result:  1.0 time:  26712 start Q:  [[1.58224561 2.49359334]
 [1.19724767 2.04780326]]
done:  0.1 1.1 2 result:  1.0 time:  30416 start Q:  [[1.46678902 2.26730893]
 [1.55958404 1.56456932]]
done:  0.1 1.1 3 result:  1.0 time:  25630 start Q:  [[3.24594436 2.13708035]
 [2.01942947 3.50110631]]
done:  0.1 1.1 4 result:  1.0 time:  27320 start Q:  [[3.99398778 3.82034082]
 [1.35017058 1.47929491]]
done:  0.1 1.1 5 result:  1.0 time:  28379 start Q:  [[2.40792221 1.95980078]
 [3.52962617 3.9782012 ]]
done:  0.1 1.1 6 result:  1.0 time:  26546 start Q:  [[3.07808419 2.86609225]
 [1.68661885 2.09640061]]
done:  0.1 1.1 7 result:  1.0 time:  26253 start Q:  [[1.39371142 1.47933309]
 [3.95900435 3.102685  ]]
done:  0.1 1.1 8 result:  1.0 time:  30071 start Q:  [[3.86109349 1.59339246]
 [3.50480534 1.8257733 ]]
done:  0.1 1.1 9 result:  1.0 time:  28988 start Q:  [[1.3058251

In [ ]:
np.mean(converged_to,2)

In [ ]:
conv_time

In [ ]:
converged_to

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(np.mean(conv_time,2), cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
np.array([random()*(np.max(R)-np.min(R))+np.min(R),random()*(np.max(R)-np.min(R))+min(min(R))])



In [ ]:
del pool # can't save a pool object
import dill
dill.dump_session('notebook_env.db')